# This code takes the ROIs generated from running for_Rose_4.ijm (a macro script for ImageJ), then converts the Image J ROIs to .csvs, then reformats the .csvs to be compatible with my calc_Area_X_lesion_volume.ipynb analysis code.

## Convert from ROI files to a combined .csv file

In [15]:
import os
import pandas as pd
from shapely.geometry import Polygon
import read_roi

def make_list_of_files_in_folder(folder_path, filetype):
    list_of_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith(filetype):
            list_of_files.append(filename)
    return list_of_files

# Dictionary to store data for CSV
for_csv_dict = {"slide_name": [], "filename": [], "coordinates": [], "region": []}

# Folder path and file type
folder_path = "/Volumes/LaCie/Microscope_images_from_glab/Area_X_Nissl_Images/No_visible_Area_X/USA5468_072124.01.02_Nissl_5x/072124.02_right_hemisphere/ROIs/"
all_roi_files = make_list_of_files_in_folder(folder_path, ".roi")

# Extract the directory name to use for the CSV file name
directory_name = os.path.basename(folder_path)
csv_filename = f"{directory_name}.csv"

# Process each .roi file
for ROI_file in all_roi_files:
    filepath = os.path.join(folder_path, ROI_file)
   
    region = ROI_file.split(".jpg_")[1]
    print(region.replace(".roi", ""))
   
    # Read the .roi file
    roi = read_roi.read_roi_file(filepath)

    for roi_name, roi_item in roi.items():
       
        # Extract coordinates and ensure the ROI is closed
        coordinates = [(x, y) for x, y in zip(roi_item['x'], roi_item['y'])]
       
        # Populate data into the dictionary
        for_csv_dict["slide_name"].append(ROI_file)
        for_csv_dict["filename"].append(filepath)
        for_csv_dict["coordinates"].append(coordinates)
        for_csv_dict["region"].append(region)

# Convert to DataFrame and save to CSV with the extracted directory name
df = pd.DataFrame(for_csv_dict)
df.to_csv(os.path.join(folder_path, csv_filename), index=False)


Striatum
Striatum
Striatum
LMAN
Striatum
Lesion
Striatum
Lesion
Striatum
Lesion
Striatum
Lesion
Striatum
Lesion
Striatum
Lesion
Striatum
Striatum
Striatum
Striatum
Striatum
Striatum


## Convert that file to a .csv format that is compatible with calc_Area_X_lesion_volume

In [16]:
import pandas as pd
import ast
import os

# Define folder path and CSV filename
#folder_path = '/Volumes/LaCie/Microscope_images_from_glab/Area_X_Nissl_Images/USA5337_042624.03.04_Nissl_5x/042624.03_left_hemisphere/042624.03_ROIs'
csv_filename = f"{os.path.basename(folder_path)}.csv"
csv_filepath = os.path.join(folder_path, csv_filename)

# Dictionary to organize data for output CSV
rose_dict = {
    "Image Name": [],
    "Striatum_x_coords": [],
    "Striatum_y_coords": [],
    "Lesion_x_coords": [],
    "Lesion_y_coords": [],
    "Area_X_x_coords": [],
    "Area_X_y_coords": [],
    "LMAN_x_coords": [],
    "LMAN_y_coords": []
}

# Load the generated CSV file from the previous cell
df = pd.read_csv(csv_filepath)

# Define the expected regions
expected_regions = ["Area_X.roi", "Striatum.roi", "Lesion.roi", "LMAN.roi"]

# Extract unique image names
all_images_names = []
for i in range(len(df["slide_name"])):
    slide_name = df["slide_name"].iloc[i]
    image_name = slide_name.split(".jpg")[0]
    all_images_names.append(image_name)

df["image_names"] = all_images_names
unique_image_names = df["image_names"].unique()

# Process each unique image
for image_name in unique_image_names:
    img_filter = df["image_names"] == image_name
    df_img = df[img_filter]
    regions = list(df_img["region"])

    rose_dict["Image Name"].append(image_name)
    missing_items = list(set(expected_regions) - set(regions))

    # Handle missing regions
    for missing_item in missing_items:
        if missing_item == "Area_X.roi":
            rose_dict["Area_X_x_coords"].append("none present")
            rose_dict["Area_X_y_coords"].append("none present")
        elif missing_item == "Striatum.roi":
            rose_dict["Striatum_x_coords"].append("none present")
            rose_dict["Striatum_y_coords"].append("none present")
        elif missing_item == "Lesion.roi":
            rose_dict["Lesion_x_coords"].append("none present")
            rose_dict["Lesion_y_coords"].append("none present")
        elif missing_item == "LMAN.roi":
            rose_dict["LMAN_x_coords"].append("none present")
            rose_dict["LMAN_y_coords"].append("none present")

    # Extract coordinates for present regions
    for i in range(len(df_img["slide_name"])):
        coords = df_img["coordinates"].iloc[i]
        region = df_img["region"].iloc[i]

        result = ast.literal_eval(coords)  # Convert string to list of tuples
        list1, list2 = zip(*result)  # Separate x and y coordinates

        if region == "Area_X.roi":
            rose_dict["Area_X_x_coords"].append(list1)
            rose_dict["Area_X_y_coords"].append(list2)
        elif region == "Striatum.roi":
            rose_dict["Striatum_x_coords"].append(list1)
            rose_dict["Striatum_y_coords"].append(list2)
        elif region == "Lesion.roi":
            rose_dict["Lesion_x_coords"].append(list1)
            rose_dict["Lesion_y_coords"].append(list2)
        elif region == "LMAN.roi":
            rose_dict["LMAN_x_coords"].append(list1)
            rose_dict["LMAN_y_coords"].append(list2)

# Convert rose_dict to a DataFrame and save as a new CSV
df_2 = pd.DataFrame(rose_dict)
output_csv_path = os.path.join(folder_path, f"{os.path.basename(folder_path)}_v2.csv")
df_2.to_csv(output_csv_path, index=False)